In [4]:
!git clone https://github.com/Tarang-Mendhe/EE5179-Deep-Learning-for-Imaging_KLA_Project
%cd EE5179-Deep-Learning-for-Imaging_KLA_Project


Cloning into 'EE5179-Deep-Learning-for-Imaging_KLA_Project'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 83 (delta 45), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 15.09 MiB | 19.01 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/EE5179-Deep-Learning-for-Imaging_KLA_Project/EE5179-Deep-Learning-for-Imaging_KLA_Project


In [21]:
'''

'''
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader

import json

import numpy as np
# from 'model_folder' import model
from best_model_arch import UNet

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import save_image

def calculate_psnr( prediction,target,mask,only_defect):


    if only_defect:
        mask = mask.to(torch.bool)
        mask_target = target[mask]
        mask_prediction = prediction[mask]
    else:
        mask_prediction=prediction
        mask_target=target
    # print("b",mask_prediction.shape)

    mse = F.mse_loss(mask_prediction, mask_target)

    psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))  # Normalizing by 1.0 since images are in [0, 1] range
    return psnr.item()

class SSIM(nn.Module):
    """Layer to compute the SSIM loss between a pair of images"""
    def __init__(self):
        super(SSIM, self).__init__()
        self.mu_x_pool   = nn.AvgPool2d(3, 1)
        self.mu_y_pool   = nn.AvgPool2d(3, 1)
        self.sig_x_pool  = nn.AvgPool2d(3, 1)
        self.sig_y_pool  = nn.AvgPool2d(3, 1)
        self.sig_xy_pool = nn.AvgPool2d(3, 1)

        self.refl = nn.ReflectionPad2d(1)

        self.C1 = 0.01 ** 2
        self.C2 = 0.03 ** 2

    def forward(self, prediction,target,mask,only_defect):

        x = self.refl(target)
        y = self.refl(prediction)

        mu_x = self.mu_x_pool(x)
        mu_y = self.mu_y_pool(y)

        sigma_x  = self.sig_x_pool(x ** 2) - mu_x ** 2
        sigma_y  = self.sig_y_pool(y ** 2) - mu_y ** 2
        sigma_xy = self.sig_xy_pool(x * y) - mu_x * mu_y

        SSIM_n = (2 * mu_x * mu_y + self.C1) * (2 * sigma_xy + self.C2)
        SSIM_d = (mu_x ** 2 + mu_y ** 2 + self.C1) * (sigma_x + sigma_y + self.C2)
        if only_defect:
         return (torch.clamp((1 - SSIM_n / SSIM_d) / 2, 0, 1))*mask
        else:
            return (torch.clamp((1 - SSIM_n / SSIM_d) / 2, 0, 1))



class DenoisingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Collect image paths in organized structure
        self._collect_image_paths()

    def _collect_image_paths(self):

        for category in os.listdir(self.root_dir):
            category_path = os.path.join(self.root_dir, category)

            if not os.path.isdir(category_path):
                continue

            for dataset_type in ['Test']:
                dataset_dir = os.path.join(category_path, dataset_type)

                if not os.path.isdir(dataset_dir):
                    continue

                gt_clean_dir = os.path.join(dataset_dir, 'Clean')
                defect_mask_dir = os.path.join(dataset_dir, 'GT_mask')
                degraded_image_dir = os.path.join(dataset_dir, 'Noise')

                # Go through each type subfolder (e.g., "type1", "type2")
                for type_subfolder in os.listdir(gt_clean_dir):
                    type_clean_dir = os.path.join(gt_clean_dir, type_subfolder)
                    type_defect_mask_dir = os.path.join(defect_mask_dir, type_subfolder)
                    type_degraded_dir = os.path.join(degraded_image_dir, type_subfolder)

                    if not (os.path.isdir(type_clean_dir) and os.path.isdir(type_defect_mask_dir) and os.path.isdir(type_degraded_dir)):
                        continue

                    # Collect images within each type folder
                    for file in os.listdir(type_clean_dir):
                        if file.endswith('.png') or file.endswith('.jpg'):
                            # Paths for GT_clean, defect mask, and degraded image
                            gt_clean_path = os.path.join(type_clean_dir, file)
                            base_name, _ = os.path.splitext(file)

                            # Construct corresponding defect mask and degraded image paths
                            defect_mask_file = f"{base_name}_mask.png"
                            defect_mask_path = os.path.join(type_defect_mask_dir, defect_mask_file)

                            degraded_image_path = os.path.join(type_degraded_dir, file)

                            # Add to data list only if all paths exist
                            if os.path.exists(defect_mask_path) and os.path.exists(degraded_image_path):
                                self.data.append({
                                    'file_name':file,
                                    'categories':category,
                                    'gt_clean': gt_clean_path,
                                    'defect_mask': defect_mask_path,
                                    'degraded_image': degraded_image_path
                                })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns the GT_clean_image, Defect_mask, and Degraded_image for the given index
        """
        paths = self.data[idx]
        filenames=paths['file_name']
        catg=paths['categories']
        gt_clean_img = cv2.imread(paths['gt_clean'])
        defect_mask_img = cv2.imread(paths['defect_mask'])
        degraded_img = cv2.imread(paths['degraded_image'])

        # Convert images to RGB
        gt_clean_img = cv2.cvtColor(gt_clean_img, cv2.COLOR_BGR2RGB)
        defect_mask_img = cv2.cvtColor(defect_mask_img, cv2.COLOR_BGR2RGB)
        degraded_img = cv2.cvtColor(degraded_img, cv2.COLOR_BGR2RGB)

        degraded_img = cv2.resize(degraded_img, (256, 256), interpolation=cv2.INTER_AREA)
        gt_clean_img = cv2.resize(gt_clean_img, (256, 256), interpolation=cv2.INTER_AREA)
        defect_mask_img = cv2.resize(defect_mask_img, (256, 256), interpolation=cv2.INTER_AREA)



        if self.transform:
            gt_clean_img = self.transform(gt_clean_img)
            defect_mask_img = self.transform(defect_mask_img)
            degraded_img = self.transform(degraded_img)
        else:
            # Default to converting to PyTorch tensors
            gt_clean_img = torch.tensor(gt_clean_img).permute(2, 0, 1).float() / 255.0
            defect_mask_img = torch.tensor(defect_mask_img).permute(2, 0, 1).float() / 255.0
            degraded_img = torch.tensor(degraded_img).permute(2, 0, 1).float() / 255.0

        return {
            'file_name':filenames,
            'categories':catg,
            'gt_clean': gt_clean_img,
            'defect_mask': defect_mask_img,
            'degraded_image': degraded_img
        }


dataset_root = "/content/drive/MyDrive/Project_evaluation_6th_Nov/Denoising_Dataset_Test_PSNR_SSIM" # Give root directory of your dataset


transform = None  # Add any custom transformations if needed

output_dir= "/content/drive/MyDrive/Project_evaluation_6th_Nov/Outputs"  ##### GIVE OUTPUT DIRECTORY PATH for SAVING IMAGES ###############

## ######################### instantiate your model and load the weights HERE  #########

model_weights_path = "/content/drive/MyDrive/best_model.pth"           # location in collab , where model is present
## ######################### instantiate your model and load the weights HERE  #########
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_classes = 3  # Specify the number of classes for segmentation
model = UNet(n_class =n_classes)
model.load_state_dict(torch.load(model_weights_path, map_location=device))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()



# Instantiate the dataset
dataset = DenoisingDataset(dataset_root, transform=transform)

# Create a DataLoader for the dataset
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

psnr_values = {}
ssim_values = {}
calculate_ssim=SSIM()

count=0
for batch in dataloader:
    filename=batch['file_name'][0]
    category=batch['categories'][0]
    gt_clean_batch = batch['gt_clean']
    defect_mask_batch = batch['defect_mask']
    degraded_image_batch = batch['degraded_image']
    count=count+1
    # print(filename)

      #####  WRITE CODE TO PASS THE INPUT TO THE MODEL #######
    with torch.no_grad():


        predicted = model(degraded_image_batch)

        #for resizing
        predicted_resized = F.interpolate(predicted, size=gt_clean_batch.shape[-2:], mode='bilinear', align_corners=False)

    ### CHANGE 'only_defect' TO 'True' FOR CALCULATING METRIC FOR DEFECT REGION ONLY
    psnr_value = calculate_psnr(predicted_resized, gt_clean_batch, defect_mask_batch,only_defect=False)
    psnr_values[category] = psnr_values.get(category, []) + [psnr_value]

    ssim_value=calculate_ssim(predicted_resized, gt_clean_batch, defect_mask_batch,only_defect=False)


    ssim_value = 1 - (ssim_value).mean()

    ssim_values[category] = ssim_values.get(category, []) + [ssim_value]

    #### SAVING IMAGES #######################
    if count%5==0:
        save_path=os.path.join(output_dir,f"{category}_{count}_{filename}")


        concatenated_image=torch.cat([degraded_image_batch[0],predicted_resized[0],gt_clean_batch[0]],dim=2)


        save_image( concatenated_image,save_path)



# Calculate averages per category and total averages
avg_psnr = {category: float(np.mean(psnr)) for category, psnr in psnr_values.items()}
mean_ssim = {category: float(np.mean(ssim)) for category, ssim in ssim_values.items()}
print("average_psnr_catg ===", avg_psnr)
print("average_SSIM_catg ===", mean_ssim)

Total_avg_psnr = float(sum(avg_psnr.values()) / len(avg_psnr))
Total_avg_ssim = float(sum(mean_ssim.values()) / len(mean_ssim))
print("TOTAL_AVG_PSNR ===", Total_avg_psnr)
print("TOTAL_AVG_SSIM ===", Total_avg_ssim)

# Organize data in a single dictionary for JSON serialization
results1 = {
    "average_psnr_per_category": avg_psnr,
    "average_ssim_per_category": mean_ssim,
    "total_average_psnr": Total_avg_psnr,
    "total_average_ssim": Total_avg_ssim
}

# Write the results to a JSON file with indentation for readability
with open('Avg_metrics.json', 'w') as fp:
    json.dump(results1, fp, indent=4)

print("Results saved to Avg_metrics.json")

<ipython-input-21-1f0a4c73cb9d>:191: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_weights_path, map_location=device))


average_psnr_catg === {'metal_nut': 18.632296562194824, 'zipper': 13.992872047424317, 'bottle': 11.948722145774148, 'pill': 15.029626039358286, 'carpet': 19.23450756072998, 'hazelnut': 20.919924302534625, 'grid': 18.025960710313583, 'toothbrush': 15.769021987915039, 'wood': 18.66476567586263, 'cable': 16.48510316212972, 'tile': 19.693086079188756, 'capsule': 12.411510467529297, 'transistor': 19.615981783185685, 'leather': 25.82487201690674, 'screw': 12.62409496307373}
average_SSIM_catg === {'metal_nut': 0.8218196630477905, 'zipper': 0.7950751185417175, 'bottle': 0.8794946670532227, 'pill': 0.819138765335083, 'carpet': 0.6402596235275269, 'hazelnut': 0.9226533770561218, 'grid': 0.7801556587219238, 'toothbrush': 0.7585717439651489, 'wood': 0.8152282238006592, 'cable': 0.857848048210144, 'tile': 0.7807979583740234, 'capsule': 0.9227038621902466, 'transistor': 0.8854418992996216, 'leather': 0.8002327680587769, 'screw': 0.9293556213378906}
TOTAL_AVG_PSNR === 17.258156366941424
TOTAL_AVG_SSI

In [27]:

# from 'model_folder' import model
from best_model_arch import UNet
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image

class PairedImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Root directory containing the 'Noise' and 'Clean' subdirectories.
            transform (callable, optional): Optional transform to apply to both images.
        """
        self.root_dir = root_dir
        self.noise_dir = os.path.join(root_dir, 'Noise')
        self.gt_dir = os.path.join(root_dir, 'Clean')
        self.transform = transform

        # List of all image files in the Noise directory
        self.image_files = sorted(os.listdir(self.noise_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Get the filename and construct the paths for both Noise and GT images
        filename = self.image_files[idx]
        noise_path = os.path.join(self.noise_dir, filename)
        gt_path = os.path.join(self.gt_dir, filename)

        # Load images
        noise_image = cv2.imread(noise_path)
        gt_image = cv2.imread(gt_path)

        # Convert BGR (OpenCV format) to RGB
        noise_image = cv2.cvtColor(noise_image, cv2.COLOR_BGR2RGB)
        gt_image = cv2.cvtColor(gt_image, cv2.COLOR_BGR2RGB)

        noise_image = cv2.resize(noise_image, (256, 256), interpolation=cv2.INTER_AREA)
        gt_image = cv2.resize(gt_image, (256, 256), interpolation=cv2.INTER_AREA)
        #defect_mask_img = cv2.resize(defect_mask_img, (256, 256), interpolation=cv2.INTER_AREA)

        # Apply transformations
        if self.transform:
            noise_image = self.transform(noise_image)
            gt_image = self.transform(gt_image)
        else:
            # Default transformation: Convert to PyTorch tensors and normalize to [0,1]
            noise_image = torch.tensor(noise_image).permute(2, 0, 1).float() / 255.0
            gt_image = torch.tensor(gt_image).permute(2, 0, 1).float() / 255.0

        return {'noise': noise_image, 'gt': gt_image, 'filename': filename}


dataset_root = "/content/drive/MyDrive/Project_evaluation_6th_Nov/Denoising_Dataset_Test_Visual"  # Update this with the actual path to your data directory

output_dir="/content/drive/MyDrive/Project_evaluation_6th_Nov/Denoising_Dataset_Test_Visual/Output"  ##### OUTPUT DIRECTORY  ###############

transform = transforms.ToTensor()  # Define any additional transformations if needed

####  CALL YOUR MODEL

model_weights_path = "/content/drive/MyDrive/best_model.pth"           # location in collab , where model is present
## ######################### instantiate your model and load the weights HERE  #########
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_classes = 3  # Specify the number of classes for segmentation
model = UNet(n_class =n_classes)
model.load_state_dict(torch.load(model_weights_path, map_location=device))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()




dataset = PairedImageDataset(root_dir=dataset_root, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)


for batch in dataloader:
    noise_images = batch['noise']
    gt_images = batch['gt']
    filenames = batch['filename']

    # predicted= ## WRITE CODE TO PASS INPUT TO THE MODEL

    with torch.no_grad():
        predicted = model(noise_images)


    save_path=os.path.join(output_dir,filenames[0])
    concatenated_image=torch.cat([noise_images[0],predicted[0],gt_images[0]],dim=2)
    save_image( concatenated_image,save_path)







<ipython-input-27-59f72cc14848>:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_weights_path, map_location=device))
